Creates a lazy xarray from a STAC collection and saves the result to a COG file

In [ ]:
!pip install boto3 rasterio stackstac pystac pystac_client numpy rioxarray python-dotenv

In [1]:
from boto3.session import Session
from rasterio.session import AWSSession
import stackstac
import pystac
import os
import rioxarray
from dotenv import load_dotenv

In [ ]:
%load_ext dotenv
%dotenv raw_stacstack_from_cos.credentials
%dotenv raw_stacstack_from_cos.env

aws_access_key_id=os.environ.get('aws_access_key_id')
aws_secret_access_key=os.environ.get('aws_secret_access_key')
cos_endpoint_url=os.environ.get('cos_endpoint_url')
stac_endpoint=os.environ.get('stac_endpoint')
collection_id=os.environ.get('collection_id')

# cog file target location (path/)filename.cog
save_to=os.environ.get('save_to',"cogfile.cog")

# limit number of returned stac items
limit=int(os.environ.get('limit', '2'))

In [ ]:
from pystac_client import Client

client = Client.open(stac_endpoint)

s1_coll = client.get_collection(collection_id)

items = []

In [ ]:
item = next(s1_coll.get_items(), None)

i = 0

while item is not None:
    items.append(item.to_dict())
    i = i + 1
    if i >= limit:
        break
    item = next(s1_coll.get_items(), None)

In [ ]:
len(items)

In [ ]:
session = Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

In [ ]:
aws_session = AWSSession(
    session=session,
    endpoint_url=cos_endpoint_url,
)

In [ ]:
data_array = stackstac.stack(items, epsg=4326, resolution=0.000089831528412, gdal_env=stackstac.DEFAULT_GDAL_ENV.updated(always=dict(session=aws_session)))

In [ ]:
data_array

In [ ]:
data_array[0,0,0,0]

In [ ]:
data_array[0,0,21380-1,57109-1]

In [ ]:
import time

start = time.time()

for index in range(57109):
    data_array[0,0,0,index]
    
end = time.time()
print(end-start)